## 251 HW9 - ImageNet Multinode DDP (Node 1)

#### Import Modules

In [1]:
!pip install wandb -qqq

In [2]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from torch.cuda.amp import autocast, GradScaler

from torch.nn.parallel import DistributedDataParallel as DDP

from torch.utils.data.distributed import DistributedSampler

import wandb

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Weights & Biases Setup

In [3]:
wandb.login()
wandb.init(project="251-hw9")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hsiungc. Use `wandb login --relogin` to force relogin


#### Set Random Seed

In [4]:
SEED=1

In [5]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

#### DDP Setup

In [6]:
os.environ["MASTER_ADDR"] = "54.185.111.250" # Keep as master node
os.environ["MASTER_PORT"] = "24053" # Use same port across all nodes

In [7]:
rank = 0
world_size = 2

In [8]:
dist.init_process_group(backend="nccl", rank=rank, world_size=world_size)

#### Hyperparameters & Variables

In [9]:
ARCH = 'resnet18'
EPOCHS = 1
LR = 0.001
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 50
TRAIN_BATCH=784
VAL_BATCH=784
WORKERS=2
TRAINDIR="/data/ebs/train"
VALDIR="/data/ebs/val"

In [10]:
best_acc1 = 0

#### Set CUDA Device

In [11]:
torch.cuda.device_count()

1

In [12]:
torch.cuda.current_device()

0

In [13]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [14]:
GPU = 'cuda:0'

In [15]:
cudnn.benchmark = True

### Functions

#### DDP

In [16]:
def cleanup():
    dist.destroy_process_group()

#### Training & Validation

In [17]:
def train(train_loader, model, criterion, optimizer, epoch):
    global global_step
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    scaler = GradScaler()
    
    # switch to train mode
    model.train()

    end = time.time()

    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        with autocast(dtype=torch.float16):
            output = model(images)
            loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient w/ scaler and do step
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        wandb.log({'accuracy1': acc1, 'accuracy5': acc5, 'loss': loss, 'learningrate': LR}, step=i)
        
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [18]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            wandb.log({'val_accuracy1': acc1, 'val_accuracy5': acc5, 'val_loss': loss, 'val_learningrate': LR}, step=1)
            
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)
    
    return top1.avg

#### Checkpoint & Progress

In [19]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [20]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [21]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

#### Learning Rate & Accuracy

In [22]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [23]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

#### Normalization & Transformations

In [24]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]

In [25]:
transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    #transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [26]:
transform_val = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

#### Datasets, DataSampler, & DataLoader

In [27]:
train_dataset = datasets.ImageFolder(
                        TRAINDIR, transform=transform_train)

In [28]:
train_sampler = DistributedSampler(train_dataset)

In [29]:
train_loader = torch.utils.data.DataLoader(
                        train_dataset, batch_size=TRAIN_BATCH, shuffle=False,
                        num_workers=WORKERS, pin_memory=True, sampler=train_sampler)

In [30]:
val_dataset = datasets.ImageFolder(
                        VALDIR, transform=transform_val)

In [31]:
val_sampler = DistributedSampler(val_dataset)

In [32]:
val_loader = torch.utils.data.DataLoader(
                        val_dataset, batch_size=VAL_BATCH, shuffle=False,
                        num_workers=WORKERS, pin_memory=True, sampler=val_sampler)

#### Model Setup

In [33]:
model = models.__dict__[ARCH](pretrained=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|███████████████████████████████████| 44.7M/44.7M [00:00<00:00, 217MB/s]


In [34]:
model = model.to(GPU)

In [35]:
ddp_model = DDP(model, device_ids=[0])

#### Loss & Optimizer

In [36]:
criterion = nn.CrossEntropyLoss().cuda(GPU)

In [37]:
optimizer = torch.optim.SGD(ddp_model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

In [38]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

#### Training & Validation Loop

In [39]:
for epoch in range(EPOCHS):
    # adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)

    save_checkpoint({
                'epoch': epoch + 1,
                'arch': ARCH,
                'state_dict': model.state_dict(),
                'best_acc1': best_acc1,
                'optimizer' : optimizer.state_dict(),
            }, is_best)

    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))

wandb.finish()
cleanup()

Epoch: [0][  0/818]	Time 13.095 (13.095)	Data  7.250 ( 7.250)	Loss 9.9395e-01 (9.9395e-01)	Acc@1  74.11 ( 74.11)	Acc@5  91.96 ( 91.96)
Epoch: [0][ 50/818]	Time  4.881 ( 2.765)	Data  4.445 ( 2.222)	Loss 9.4707e-01 (9.7720e-01)	Acc@1  73.98 ( 74.97)	Acc@5  92.35 ( 92.26)
Epoch: [0][100/818]	Time  4.750 ( 2.670)	Data  4.314 ( 2.180)	Loss 9.3304e-01 (9.5672e-01)	Acc@1  76.79 ( 75.72)	Acc@5  93.11 ( 92.48)
Epoch: [0][150/818]	Time  4.876 ( 2.635)	Data  4.439 ( 2.163)	Loss 7.9376e-01 (9.4291e-01)	Acc@1  78.57 ( 76.01)	Acc@5  94.90 ( 92.72)
Epoch: [0][200/818]	Time  4.749 ( 2.617)	Data  4.313 ( 2.154)	Loss 8.9460e-01 (9.3334e-01)	Acc@1  76.40 ( 76.11)	Acc@5  92.98 ( 92.85)
Epoch: [0][250/818]	Time  4.816 ( 2.610)	Data  4.380 ( 2.152)	Loss 8.4463e-01 (9.2607e-01)	Acc@1  78.44 ( 76.28)	Acc@5  92.98 ( 92.94)
Epoch: [0][300/818]	Time  4.600 ( 2.607)	Data  4.165 ( 2.153)	Loss 7.8119e-01 (9.1880e-01)	Acc@1  79.08 ( 76.41)	Acc@5  95.54 ( 93.03)
Epoch: [0][350/818]	Time  4.678 ( 2.605)	Data  4.242 ( 

accuracy1,▁▂▅▄▄▆▄▄█▅▅▇▂▆▅▇▂▅▅▆█▆▃▆▅▇▇▇▅▆▆▆▇▆▆▅▃▇▅█
accuracy5,▁▃▅▁▅▃▅▅▄▆▆▇▃▆▄▄▄▄▅▄▆▅▃▄▅▇▅▅▆▅▆▇▅▆▃▅▃▇▄█
learningrate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▇▃▆▅▆▄▄▃▄▄▂▆▄▄▃▇▅▄▄▂▃▅▃▄▁▃▃▄▄▃▃▂▃▅▄▅▃▅▂
accuracy1,67.85714
accuracy5,87.5
learningrate,0.001
loss,1.09559
